THIS IS TO FIND THE FAIRNESS AND GOODNESS FOR BITCOIN_DATA using EDGE_DATA CODE

In [1]:
import networkx as nx
import math
import pandas as pd
from prettytable import PrettyTable
import csv

def initialize_scores(G):
    fairness = {}
    goodness = {}
    
    nodes = G.nodes()
    for node in nodes:
        fairness[node] = 1
        try:
            goodness[node] = G.in_degree(node, weight='weight') * 1.0 / G.in_degree(node)
        except:
            goodness[node] = 0
    return fairness, goodness

def compute_fairness_goodness(G):
    fairness, goodness = initialize_scores(G)
    
    nodes = G.nodes()
    iter = 0
    while iter < 100:
        df = 0
        dg = 0

        print('-----------------')
        print("Iteration number", iter)
        
        print('Updating goodness')
        for node in nodes:
            inedges = G.in_edges(node, data='weight')
            g = 0
            for edge in inedges:
                if isinstance(edge[2], float):
                    g += fairness[edge[0]] * edge[2]
                else:
                    g += fairness[edge[0]] * edge[2]["weight"]

            try:
                dg += abs(g / len(inedges) - goodness[node])
                goodness[node] = g / len(inedges)
            except:
                pass

        print('Updating fairness')
        for node in nodes:
            outedges = G.out_edges(node, data='weight')
            f = 0
            for edge in outedges:
                if isinstance(edge[2], float):
                    f += 1.0 - abs(edge[2] - goodness[edge[1]]) / 2.0
                else:
                    f += 1.0 - abs(edge[2]["weight"] - goodness[edge[1]]) / 2.0
            try:
                df += abs(f / len(outedges) - fairness[node])
                fairness[node] = f / len(outedges)
            except:
                pass
        
        print('Differences in fairness score and goodness score = %.2f, %.2f' % (df, dg))
        if df < math.pow(10, -6) and dg < math.pow(10, -6):
            break
        iter += 1
    
    return fairness, goodness

G = nx.DiGraph()

with open("bitcoin.csv", "r") as f:
    csvreader = filter(lambda row: row[2].replace(".", "").replace("-", "").isnumeric(), csv.reader(f))
    for row in csvreader:
        G.add_edge(row[0], row[1], weight=float(row[2]))

fairness, goodness = compute_fairness_goodness(G)
fairness_df = pd.DataFrame.from_dict(fairness, orient='index', columns=['fairness'])
goodness_df = pd.DataFrame.from_dict(goodness, orient='index', columns=['goodness'])
result_df = pd.concat([fairness_df, goodness_df], axis=1)

x = PrettyTable()
x.field_names = ["Node", "Fairness", "Goodness"]
for i in range(result_df.shape[0]):
    x.add_row([result_df.index[i], result_df.fairness[i], result_df.goodness[i]])
print(x)


-----------------
Iteration number 0
Updating goodness
Updating fairness
Differences in fairness score and goodness score = 2227.88, 0.00
-----------------
Iteration number 1
Updating goodness
Updating fairness
Differences in fairness score and goodness score = 1375.76, 5277.08
-----------------
Iteration number 2
Updating goodness
Updating fairness
Differences in fairness score and goodness score = 1530.29, 3677.01
-----------------
Iteration number 3
Updating goodness
Updating fairness
Differences in fairness score and goodness score = 2150.73, 4919.58
-----------------
Iteration number 4
Updating goodness
Updating fairness
Differences in fairness score and goodness score = 3561.14, 7563.16
-----------------
Iteration number 5
Updating goodness
Updating fairness
Differences in fairness score and goodness score = 7064.23, 13603.82
-----------------
Iteration number 6
Updating goodness
Updating fairness
Differences in fairness score and goodness score = 16031.09, 28006.72
-------------

In [2]:
highest_fairness_value = max(fairness.values())
lowest_fairness_value = min(fairness.values())

highest_fairness_nodes = [node for node, value in fairness.items() if value == highest_fairness_value]
lowest_fairness_nodes = [node for node, value in fairness.items() if value == lowest_fairness_value]

print("Nodes with highest fairness:", highest_fairness_nodes)
print("Fairness value:", highest_fairness_value)
print("Nodes with lowest fairness:", lowest_fairness_nodes)
print("Fairness value:", lowest_fairness_value)

print("\n")

highest_goodness_value = max(goodness.values())
lowest_goodness_value = min(goodness.values())

highest_goodness_nodes = [node for node, value in goodness.items() if value == highest_goodness_value]
lowest_goodness_nodes = [node for node, value in goodness.items() if value == lowest_goodness_value]

print("Nodes with highest goodness:", highest_goodness_nodes)
print("Goodness value:", highest_goodness_value)
print("Nodes with lowest goodness:", lowest_goodness_nodes)
print("Goodness value:", lowest_goodness_value)

Nodes with highest fairness: ['1028', '2367', '2296', '3421', '3418', '3402', '3385', '3367', '3349', '3339', '3335', '3317', '3313', '3295', '3283', '3256', '3259', '3247', '2368', '3215', '3202', '3163', '3148', '3124', '3113', '3013', '3103', '3101', '3083', '3084', '3078', '3062', '3059', '2273', '3048', '3004', '3006', '3000', '2999', '2989', '2941', '2947', '2945', '2926', '2920', '2782', '2904', '2898', '2881', '2858', '2837', '2830', '2632', '2810', '2793', '2786', '2783', '2775', '2772', '2771', '2767', '2764', '2763', '2752', '2750', '2747', '2744', '2743', '2707', '2713', '2700', '2695', '2619', '2605', '7348', '7425', '7557', '767', '469', '268', '2839', '1929', '7401', '1824', '2903', '3049', '2024', '1634', '2997', '2868', '2740', '2709', '1176', '7559', '7480', '7477', '7485', '1395', '41', '3365', '3192', '1499', '2307', '3351', '3177', '3052', '407', '7567', '7574', '7484', '7505', '1896', '3400', '7352', '7362', '7361', '7359', '7360', '7358', '7496', '7382', '7357', 